

```
The Notebook can be loaded from this link:
https://colab.research.google.com/drive/11TtbhN4nuvCMn-VqsL65Cf9AZL8yx-yv
```



In [1]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/chengmar09/comp0053.git  

GitHub user··········
GitHub password··········
Cloning into 'comp0053'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 40 (delta 16), reused 28 (delta 8), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [2]:
!echo 'Check Current Dir'
%cd /content/comp0053/
!pwd
!ls
!pip install -r Requirement.txt
!pip install h5py

Check Current Dir
/content/comp0053
/content/comp0053
COMP0053_EmoPain_Demo.ipynb  helper.py	Requirement.txt
data			     README.md	train.py
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=c2f10b934c72825bcd7d2fbde997c7b93ddda04c7cc222df7a4d465fb84744e4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
from helper import *

movementDataLoader = DataReader('Movementdata')
#train_data, valid_data = movementDataLoader.get_data()
(train_x, train_y), (valid_x, valid_y) = movementDataLoader.more_more_processing() #<- Automatically download data
movementDataLoader.get_weights(data='Movementdata', name='stack_bilstm.h5')

Using TensorFlow backend.


Create a ->Movementdata<- folder 

Now downloading training data

Now downloading Validation data
Unzipping training data
Unzipping validation data
There is total of 23 training data
There is total of 12 Validation data
Downlading weights...


In [4]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, GRU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1_m])

# checkpoint
checkpoint_path = "stack_bilstm.h5"

model = load_model(checkpoint_path, custom_objects={'f1_m': f1_m})
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

history = model.fit(train_x, train_y, validation_split=0.1, epochs=1, callbacks=callbacks_list)  # starts training







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 320496 samples, validate on 35611 samples
Epoch 1/1
320496/320496 [==============================] - 1575s 5ms/step - loss: 0.1987 - acc: 0.9192 - f1_m: 0.7525 - val_loss: 0.7470 - val_acc: 0.7787 - val_f1_m: 0.0382

Epoch 00001: val_loss improved from inf to 0.74699, saving model to stack_bilstm.h5


In [5]:
from keras.models import load_model

model = load_model(checkpoint_path, custom_objects={'f1_m': f1_m})
# summarize model.
model.summary()

#model.load_weights('best_model.hdf5')
validation_score = model.evaluate(valid_x, valid_y)

for name, score in zip(model.metrics_names, validation_score):
    print(f'{name} : {score}')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 13, 256)           134144    
_________________________________________________________________
dropout_4 (Dropout)          (None, 13, 256)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 13, 256)           394240    
_________________________________________________________________
dropout_5 (Dropout)          (None, 13, 256)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                